In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import torch
import random

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
q_walk = np.zeros((7,7))
r_walk = np.zeros((7, 7))
r_walk[-1, -1] = 1

In [4]:
done = False
start = 4
prob = 0.5

for i in range(100):
    done = False
    start = 4
    prob = 0.5
    while not done:
        rand = random.random()
        if rand >= 0.5:
            next_step = start + 1
        else:
            next_step = start - 1
        if (next_step == -1) or (next_step == 7):
            done = True
            continue
        q_walk[start, next_step] = q_walk[start, next_step] + r_walk[next_step, next_step] + 0.5*max(q_walk[next_step]) - q_walk[start, next_step]

        start = next_step

In [5]:
q_walk

array([[0.        , 0.04166667, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.02083333, 0.        , 0.08333333, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.04166667, 0.        , 0.16666667, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.08333333, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.16666667, 0.        ,
        0.66666667, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.33333333,
        0.        , 1.33333333],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.66666667, 0.        ]])

In [6]:
# solve gridworld problem as Markov reward process

In [7]:
v = torch.tensor(
    (
        (1, -1/2, 0, 0, 0, 0, 0, 0), 
        (1/2, -1, 1/2, 0, 0, 0, 0, 0),
        (0, 1/2, -1, 1/2, 0, 0, 0, 0),
        (0, 0, 1/3, -1, 1/3, 0, 0, 0),
        (0, 0, 0, 1/3, -1, 0, 1/3, 0),
        (0, 0, 0, 0, 0, -1, 1/2, 0),
        (0, 0, 0, 0, 1/3, 1/3, -1, 1/3),
        (0, 0, 0, 0, 0, 0, 1/2, -1)
    )
)

In [8]:
v

tensor([[ 1.0000, -0.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5000, -1.0000,  0.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.5000, -1.0000,  0.5000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.3333, -1.0000,  0.3333,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.3333, -1.0000,  0.0000,  0.3333,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0000,  0.5000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.3333,  0.3333, -1.0000,  0.3333],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5000, -1.0000]])

In [9]:
s = torch.tensor((0, 0, 0, -1/3, 1/3, 0, 0, 1/2))

In [10]:
s

tensor([ 0.0000,  0.0000,  0.0000, -0.3333,  0.3333,  0.0000,  0.0000,  0.5000])

In [11]:
torch.inverse(v)

tensor([[ 1.5676, -1.1351, -0.7027, -0.4054, -0.1622, -0.0270, -0.0811, -0.0270],
        [ 1.1351, -2.2703, -1.4054, -0.8108, -0.3243, -0.0541, -0.1622, -0.0541],
        [ 0.7027, -1.4054, -2.1081, -1.2162, -0.4865, -0.0811, -0.2432, -0.0811],
        [ 0.2703, -0.5405, -0.8108, -1.6216, -0.6486, -0.1081, -0.3243, -0.1081],
        [ 0.1081, -0.2162, -0.3243, -0.6486, -1.4595, -0.2432, -0.7297, -0.2432],
        [ 0.0270, -0.0541, -0.0811, -0.1622, -0.3649, -1.3108, -0.9324, -0.3108],
        [ 0.0541, -0.1081, -0.1622, -0.3243, -0.7297, -0.6216, -1.8649, -0.6216],
        [ 0.0270, -0.0541, -0.0811, -0.1622, -0.3649, -0.3108, -0.9324, -1.3108]])

In [12]:
vinv = torch.linalg.inv(v)

In [13]:
v = np.array(
    (
        (1, -1/2, 0, 0, 0, 0, 0, 0), 
        (1/2, -1, 1/2, 0, 0, 0, 0, 0),
        (0, 1/2, -1, 1/2, 0, 0, 0, 0),
        (0, 0, 1/3, -1, 1/3, 0, 0, 0),
        (0, 0, 0, 1/3, -1, 0, 1/3, 0),
        (0, 0, 0, 0, 0, -1, 1/2, 0),
        (0, 0, 0, 0, 1/3, 1/3, -1, 1/3),
        (0, 0, 0, 0, 0, 0, 1/2, -1)
    )
)

In [14]:
s = np.array((0, 0, 0, -1/3, 1/3, 0, 0, 1/2))

In [15]:
np.linalg.inv(v).dot(s)

array([ 0.06756757,  0.13513514,  0.2027027 ,  0.27027027, -0.39189189,
       -0.22297297, -0.44594595, -0.72297297])

In [16]:
# check this answer by doing updates at each step

In [17]:
q_walk = np.zeros((7,7))
r_walk = np.zeros((7, 7))
r_walk[-1, -1] = 1

In [18]:
done = False
start = 4
prob = 0.5

for i in range(100):
    done = False
    start = 4
    prob = 0.5
    while not done:
        rand = random.random()
        if rand >= 0.5:
            next_step = start + 1
        else:
            next_step = start - 1
        if (next_step == -1) or (next_step == 7):
            done = True
            continue
        q_walk[start, next_step] = q_walk[start, next_step] + r_walk[next_step, next_step] + 0.5*max(q_walk[next_step]) - q_walk[start, next_step]

        start = next_step

In [19]:
r = np.array((0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0))
p = np.array(
    (
        (0, 1/2, 0, 0, 0, 0, 0, 0, 1/2, 0, 0), # start
        (1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0), # A
        (0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0), # B
        (0, 0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0), # C
        (0, 0, 0, 1/3, 0, 1/3, 1/3, 0, 0, 0, 0), # D
        (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0), # end 1
        (0, 0, 0, 0, 1/3, 0, 0, 1/3, 0, 1/3, 0), # E
        (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0), # end -1
        (1/2, 0, 0, 0, 0, 0, 0, 0, 0, 1/2, 0), # F
        (0, 0, 0, 0, 0, 0, 1/3, 0, 1/3, 0, 1/3), # G
        (0, 0, 0, 0, 0, 0, 0, 1/2, 0, 1/2, 0), # H
    )
)

v = np.zeros((11))

In [20]:
for i in range(100):
    done = False
    state = 0
    for i in range(5):
        next_state = np.where(p[state] > 0)
        print(next_state)
        for j in next_state:
            print(p[j])
#             v[state] = v[state] + (p[state, next_state] * (r[next_state] + v[next_state]))
#             state = next_state

(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.

(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]
(array([1, 8]),)
[[0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.

In [21]:
v

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])